# 画像表示用の基本コード

In [ ]:
import cv2

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

#画像をWindowで表示
cv2.imshow('image',img)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# スケーリング

In [ ]:
import cv2

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

#方法1:縦横の倍数
res = cv2.resize(img,None,fx=1.2, fy=1.5)
#方法2：縦横のピクセル数
res2 = cv2.resize(img,(300,300), interpolation = cv2.INTER_CUBIC)

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result1',res)
cv2.imshow('result2',res2)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# 並進

In [ ]:
import cv2
import numpy as np

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

height,width,_=img.shape

#並進に関するパラメータ
M = np.float32([[1,0,50],[0,1,100]])
#warpAffineで移動させる
dst = cv2.warpAffine(img,M,(width+100,height+100))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# 回転

In [ ]:
import cv2
import numpy as np

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

height,width,_=img.shape

#回転に関するパラメータ
M = cv2.getRotationMatrix2D((width/2,height/2),90,1)
#warpAffineで回転させる
dst = cv2.warpAffine(img,M,(width+100,height+100))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# アフィン変換

In [ ]:
import cv2
import numpy as np

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)

height,width,_=img.shape

#移動させたい点
pts1 = np.float32([[0,0],[width,0],[0,height]])
#移動先
pts2 = np.float32([[0,0],[width,50],[50,height]])

img2=img.copy()

for i in range(3):
    img2=cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])),10,(0,0,255),-1)

for i in range(3):
    img2=cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])),4,(255,0,0),-1)


#アフィン変換(変形)のパラメータ
M = cv2.getAffineTransform(pts1,pts2)
#warpAffineで変形
dst = cv2.warpAffine(img,M,(width+100,height+100))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# 射影変換

In [ ]:
import cv2
import numpy as np

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)

height,width,_=img.shape

#移動させたい点
pts1 = np.float32([[0,0],[width,0],[width,height],[0,height]])
#移動先
pts2 = np.float32([[30,40],[width-100,30],[width-10,height-50],[20,height-30]])

img2=img.copy()

for i in range(4):
    img2=cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])),10,(0,0,255),-1)

for i in range(4):
    img2=cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])),5,(255,0,0),-1)


#射影変換(変形)のパラメータ
M = cv2.getPerspectiveTransform(pts1,pts2)
#warpAffineで変形
dst = cv2.warpPerspective(img,M,(width+100,height+100))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/marker.jpg',1)

height,width,_=img.shape

#移動先
pts2 = np.float32([[0,0],[width,0],[width,height],[0,height]])
#移動させたい点
pts1 = np.float32([[100,68],[218,58],[263,168],[114,193]])

img2=img.copy()

for i in range(4):
    img2=cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])),10,(0,0,255),-1)

for i in range(4):
    img2=cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])),5,(255,0,0),-1)


#射影変換(変形)のパラメータ
M = cv2.getPerspectiveTransform(pts1,pts2)
#warpPerspectiveで変形
dst = cv2.warpPerspective(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)